# Data Wrangling: Join, Combine and Reshape

## Hierarcihcal Indexing

Hierarchical indexing enables multiple index levels on axis. Anther way to look at it is as working with higher dim data in lower dim form. 

In [1]:
import pandas as pd
import numpy as np

In [9]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.731638
   2    0.919470
   3    0.254580
b  1    0.105696
   3    0.771312
c  1    0.946652
   2    0.644269
d  2    0.753497
   3    0.321431
dtype: float64

In [10]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [11]:
data["b"]

1    0.105696
3    0.771312
dtype: float64

In [12]:
data["b":"c"]

b  1    0.105696
   3    0.771312
c  1    0.946652
   2    0.644269
dtype: float64

In [13]:
data[["b", "d"]]

b  1    0.105696
   3    0.771312
d  2    0.753497
   3    0.321431
dtype: float64

In [14]:
# Selection is even possible from am inner level
data.loc[:, 2]

a    0.919470
c    0.644269
d    0.753497
dtype: float64

In [15]:
# For example data can be transformed to dataframe using unstack
data.unstack()

,1,2,3
a,0.731638,0.919470,0.254580
b,0.105696,NaN,0.771312
c,0.946652,0.644269,NaN
d,NaN,0.753497,0.321431


In [16]:
# The inverse operation of unstack i stack
data.unstack().stack()

a  1    0.731638
   2    0.919470
   3    0.254580
b  1    0.105696
   3    0.771312
c  1    0.946652
   2    0.644269
d  2    0.753497
   3    0.321431
dtype: float64

In [17]:
# With dataframe either axis can hierarchical index
frame = pd.DataFrame(np.arange(12).reshape((4,3)), 
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [19]:
# The hierarchical levels can have names.
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame
# Names superseeds name attribute which can be used in single level indexes

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
# Number of levels is returned by
frame.index.nlevels

2

In [21]:
# Same selection logic can be applied to columns as for indexes
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [23]:
# A multiindex can be created by itself and then reused
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                         ["Green", "Red", "Green"]],
                         names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reorderong and Sorting Levels

Sometimes order of levels need to be rearanged and sometimes ordering on a specific level is needed. The `swaplevel` method takes two level numbers or names and returns new object with levels interchanged

In [26]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` by default sorts on all levels lexiographically but it also acceps parameter level to sort only by one leve

In [27]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [28]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

In [30]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [34]:
frame.groupby(level="color", axis=1).sum()

/tmp/ipykernel_4495/3552194085.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis=1).sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [36]:
frame.T.groupby(level="color").sum()

key1   a      b    
key2   1  2   1   2
color              
Green  2  8  14  20
Red    1  4   7  10

## Indexing with a DataFrame's columns

In [37]:
# It is not unusual to move one or two columns of dataframe as the row index. Sometimes and index is moved to be a dataframe column.
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                             "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [38]:
# Set index will create index of columns
frame.set_index(["c", "d"])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [39]:
# By default columns are removed from dataframe but this can be changed
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [40]:
# We can reset it 
frame.set_index(["c", "d"]).reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets

`pandas.merge`
Connects rows in DataFrame based on one or more keys. Same as SQL DB join.

`pandas.concat`
Concatanate or stack object together along an axis

`combine_first`
Splice together overlapping data to fill in mssing values in one object with values in another

### Database style data frames joins

In [41]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [42]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [44]:
# When column to join on is not specified pandas merge over overlapping columns names
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [45]:
# It is better practice to specify the merge on columns
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [46]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [50]:
# By default pandas does inner join, other options are: left, right and outher
# Outer takes the union of keys
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [51]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


### Merging on Indexes

In [52]:
# In some cases merging is done on indexes, left_index=True and/or right_index=True can be passed to achieve this
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [53]:
# Default merge is intersect of the keys and union can be achieved with outer
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [54]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                                "Nevada", "Nevada"],
                       "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})

righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})

lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [55]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [56]:
# Join on hierarhical index
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [58]:
# outer
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [59]:
# Using indexes on both sides
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")

left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [60]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")

right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [61]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [64]:
# Dataframe has join method that joins over the index and can be used to join multiple data frames 
# with similar indexes and non overlaping columns

left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [70]:
# Compared to pandas.merge join does left join by default 
# It also supports joining on the index of the passed df with a column of calling df
# left1
# right1
left1.join(right1, on="key")
# It can be thinked of as joining data into object whose join method was called

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [71]:
# For simple index to index merge list of dfs can be passed
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [72]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
